<a href="https://colab.research.google.com/github/abhishek1397/Drone_Detection/blob/master/Train_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [66]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Apr  9 15:08:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [67]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ln -s /content/gdrive/My\ Drive/ /mydrive/yolov3 
!ls /mydrive/yolov3


Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/yolov3/My Drive': File exists
 images.zip  'My Drive'


**1) Clone the Darknet**



In [68]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [69]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused v

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [72]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [73]:
!echo "Drone" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


**4) Download darknet53 model**

In [74]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-04-09 15:10:55--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.4’

darknet53.conv.74.4 100%[===================>] 154.96M   163KB/s    in 10m 17s 

2020-04-09 15:21:14 (257 KB/s) - ‘darknet53.conv.74.4’ saved [162482580/162482580]



**5) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be on the main folder

In [79]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990670.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990673.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990673.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990676.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990676.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990679.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990679.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990682.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990682.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990685.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990685.txt  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438990688.jpg  
  inflating: data/obj/bangkok-thailand-june-18-2016-260nw-438

In [80]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/pilsen-czech-republic-april-22-260nw-409851811.jpg', 'data/obj/dji-mavic-pro-drone-moscow-260nw-1028042260.jpg', 'data/obj/bangkok-thailand-june-18-2016-260nw-438992611.jpg', 'data/obj/kaunas-lithuania-december-30-2015-260nw-359958602.jpg', 'data/obj/varna-bulgaria-july-05-2015-260nw-359672102.jpg', 'data/obj/kagawa-japan-april-22-2017-260nw-625596302.jpg', 'data/obj/kiev-ukraine-april-24-2016-260nw-410937175.jpg', 'data/obj/fort-collins-co-usa-november-260nw-232414648.jpg', 'data/obj/kuala-lumpur-oct-14-drones-260nw-328221584.jpg', 'data/obj/udonthani-thailandmay-06-2016-drone-260nw-417484558.jpg', 'data/obj/nong-khai-thailand-jan-3-260nw-703364929.jpg', 'data/obj/perak-malaysia-20th-march-2017-260nw-638972497.jpg', 'data/obj/kota-kinabalu-sabah-august-2-260nw-689069086.jpg', 'data/obj/krasnodar-russia-may-30-2017-260nw-651799621.jpg', 'data/obj/thailandlopburi11jun2017-dji-mavic-pro-drone-260nw-657853369.jpg', 'data/obj/nong-khai-thailand-jan-3-260nw-695286682.jpg', 'data/

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [82]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 compute_capability = 370, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
